In [2]:
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

c:\Users\Win 11\Desktop\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Load and chunk your dataset
chunk_size = 300
chunk_overlap = 100

# loading data
loader = WikipediaLoader(query="Steve Jobs", load_max_docs=5)
documents = loader.load()

# text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
docs = text_splitter.split_documents(documents=documents)
docs

[Document(metadata={'title': 'Steve Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers. \nJobs saw the commercial potential 

In [6]:
# 2. Build vector store
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)

In [8]:
# 3. Set up the LLM you’ll use to generate hypothetical answers
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm = init_chat_model("openai:o4-mini")

In [9]:
from langchain.vectorstores import Chroma
## creating vector store
db = Chroma.from_documents(documents = docs,embedding=embeddings,persist_directory = "output/steve_jobs_for_hyde.db")
##create the retriever
base_retriever=db.as_retriever(search_kwargs = {"k":5})

In [10]:
from langchain_core.output_parsers import StrOutputParser
## Generating a prompt gor generating HyDE\n",
from langchain.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate
def get_hyde_doc(query):
    template = """Imagine you are an expert writing a detailed explanation on the topic: '{query}'
    create a hypothetical answer for the topic"""
    system_message_prompt = SystemMessagePromptTemplate.from_template(template = template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
    messages = chat_prompt.format_prompt(query = query).to_messages()
    print(messages)
    response = llm.invoke(messages)
    hypo_doc = response.content
    return hypo_doc

In [11]:
query = 'When was Steve Jobs fired from Apple?'
print(get_hyde_doc(query=query))

[SystemMessage(content="Imagine you are an expert writing a detailed explanation on the topic: 'When was Steve Jobs fired from Apple?'\n    create a hypothetical answer for the topic", additional_kwargs={}, response_metadata={})]
Steve Jobs’s departure from Apple wasn’t a sudden sack-in-the-office moment so much as the climax of a long, bruising power struggle. Here’s a concise, chronologically organized explanation of what happened and when:

1. Background  
   – 1976–1983: Jobs co-founded Apple with Steve Wozniak and quickly became the public face and product visionary. Early successes (Apple I, Apple II) propelled him into the company’s executive ranks.  
   – 1983: Concerned about his own management shortcomings, Jobs recruited PepsiCo executive John Sculley to be Apple’s CEO, famously asking, “Do you want to sell sugar water the rest of your life, or do you want to come with me and change the world?”  

2. The Macintosh Launch and Early Tensions  
   – January 1984: Apple unveiled

In [12]:
### Langchain-HypotheticalDocumentEmbedder
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

# Step 1: Load and split documents
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [14]:
# Step 2: Set up LLM and embeddings
base_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Win 11\AppData\Local\Temp\ipykernel_18676\2818700810.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  base_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [16]:
# Step 3: HyDE Embedder using prompt_key='web_search'
hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embeddings,
    prompt_key="web_search",
)

In [17]:
# Step 4: Store documents in Chroma with HyDE embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=hyde_embedding_function,
    persist_directory="output/langchain",
)

In [18]:
# Step 5: RAG answer generation prompt\n",
rag_prompt = PromptTemplate.from_template("""
    Use the context below to answer the question.
    Context:
    {context}
    Question: {input}
    """)
rag_chain = create_stuff_documents_chain(llm=llm, prompt=rag_prompt)

In [19]:
# Step 6: Final RAG Pipeline
def hyde_rag_pipeline(query):
    matched_docs = vectorstore.similarity_search(query, k=4)
    print(matched_docs)
    response = rag_chain.invoke({
        "input": query,
        "context": matched_docs
    })
    return response

In [20]:
# Step 7: Run example query
query = "What memory modules does LangChain provide?"
answer = hyde_rag_pipeline(query)
print("✅ Final Answer:\n", answer)

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v1)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v10)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v7)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_cont

In [22]:
from langchain.prompts import PromptTemplate
custom = PromptTemplate.from_template(
    "Generate a concise hypothetical answer for this topic: {query}")
"# Step 3: HyDE Embedder using prompt_key='web_search'\n",
hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embeddings,
    custom_prompt=custom
)
print(hyde_embedding_function)

verbose=False base_embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False) llm_chain=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='Generate a concise hypothetical answer for this topic: {query}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D3DE24F740>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D3DE2E7B6